# 02_model_training.ipynb

This notebook trains a Convolutional Neural Network (CNN) to detect pneumonia from chest X-ray images using PyTorch. It loads preprocessed data from a saved `.npz` file, defines the model architecture, and trains the model.



In [ ]:
## Imports and Setup

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
import os

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
device

device(type='cpu')

In [ ]:
# Define the path to the processed data directory
processed_data_dir = '/content/drive/MyDrive/projects/data/processed'

# Define the path for the .npz file
data_file_path = os.path.join(processed_data_dir, 'data.npz')

# Load the data
data = np.load(data_file_path)

train_images = data['train_images']
train_labels = data['train_labels']
test_images = data['test_images']
test_labels = data['test_labels']
val_images = data['val_images']
val_labels = data['val_labels']

# Display shapes
print(f"Training set: {train_images.shape}, Labels: {len(train_labels)}")
print(f"Testing set: {test_images.shape}, Labels: {len(test_labels)}")
print(f"Validation set: {val_images.shape}, Labels: {len(val_labels)}")


Training set: (4172, 150, 150, 3), Labels: 4172
Testing set: (624, 150, 150, 3), Labels: 624
Validation set: (1060, 150, 150, 3), Labels: 1060


In [ ]:
from torchvision import transforms
from PIL import Image
class PneumoniaDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

        # Convert labels from strings to integers
        label_mapping = {'NORMAL': 0, 'PNEUMONIA': 1}
        self.labels = np.array([label_mapping[label] for label in labels])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.uint8)  # Ensure uint8 format
        image = Image.fromarray(image)  # Convert numpy array to PIL Image
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)  # Apply transformations
        return image, torch.tensor(label, dtype=torch.long)  # Convert label to tensor




transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Resize to the target size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])


# Convert numpy arrays to PyTorch DataLoader
def preprocess_data(images, labels, transform=None):
    dataset = PneumoniaDataset(images, labels, transform)
    return DataLoader(dataset, batch_size=32, shuffle=True)

train_loader = preprocess_data(train_images, train_labels, transform)
val_loader = preprocess_data(val_images, val_labels, transform)
test_loader = preprocess_data(test_images, test_labels, transform)


In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm  # Importing tqdm for progress bars

def train_model(model, train_loader, val_loader, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)  # Move model to the appropriate device

    criterion = nn.CrossEntropyLoss()  # Define loss function
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define optimizer

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        # Add tqdm to training loop
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the device

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update model parameters

            running_loss += loss.item()

            # Optionally, update tqdm with current loss for the batch
            train_loader_tqdm.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        # Add tqdm to validation loop
        val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)

        with torch.no_grad():
            for images, labels in val_loader_tqdm:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the device
                outputs = model(images)  # Forward pass
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

                # Optionally, update tqdm with current validation loss
                val_loader_tqdm.set_postfix(loss=loss.item())

        val_loss = val_loss / len(val_loader)
        accuracy = 100 * correct_predictions / total_samples
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

    print("Training complete")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PneumoniaNet(nn.Module):
    def __init__(self):
        super(PneumoniaNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Calculate the input size for fc1 dynamically
        self._calculate_fc1_input_size()

        self.fc1 = nn.Linear(self.fc1_input_size, 512)  # Use calculated input size for fc1
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)

    def _calculate_fc1_input_size(self):
        # Pass a dummy input through the network to compute the size before the fully connected layers
        dummy_input = torch.zeros(1, 3, 150, 150)  # Batch size 1, 3 channels, 150x150 image
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        self.fc1_input_size = x.view(1, -1).size(1)  # Flatten the tensor and get its size

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Dynamically flatten the tensor

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Instantiate the model and move it to the appropriate device
model = PneumoniaNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# train_model function
train_model(model, train_loader, val_loader, num_epochs=10)

Epoch [1/10], Loss: 0.6145


Validation Loss: 0.5742, Accuracy: 73.87%


Epoch [2/10], Loss: 0.5759


Validation Loss: 0.5811, Accuracy: 73.87%


Epoch [3/10], Loss: 0.5757


...

Validation Loss: 0.5688, Accuracy: 73.87%


Epoch [9/10], Loss: 0.5746


Validation Loss: 0.5810, Accuracy: 73.87%


Epoch [10/10], Loss: 0.5746


Validation Loss: 0.5676, Accuracy: 73.87%
Training complete
